In [1]:
import os
import glob
import json
import pandas
from tqdm import tqdm 
from collections import defaultdict
from short_sport.video_trainer.labels import EVENT_DICTIONARY_V2
from src.preprocessing.video import(
    json_reader,
    round_down_to_minute,
    round_down_to_minute_v2,
    normalize_path
)

In [22]:
EVENT_DICTIONARY_V2

{'Penalty': 0,
 'Kick-off': 1,
 'Goal': 2,
 'Substitution': 3,
 'Offside': 4,
 'Shots on target': 5,
 'Shots off target': 6,
 'Clearance': 7,
 'Ball out of play': 8,
 'Throw-in': 9,
 'Foul': 10,
 'Indirect free-kick': 11,
 'Direct free-kick': 12,
 'Corner': 13,
 'Yellow card': 14,
 'Red card': 15,
 'Yellow->red card': 16,
 'Not Event': 17}

In [2]:
EVENT_DICT = {key: index for index, key in enumerate(EVENT_DICTIONARY_V2.keys())}
EVENT_DICT
video_path = os.listdir("F:/video_classification")
video_paths = [f"F:/video_classification/{i}" for i in video_path]
# labels = []
# sss = 0
# for i in video_path:
#     # print(i)
#     matches = glob.glob(f"F:/video_classification/{i}/Labels-v2.json")
#     if matches:  # Nếu tìm thấy ít nhất 1 file
#         labels.append(normalize_path(matches[0]))
#     else:
#         sss += 1
#         # print(f"Warning: No label file found for {i}")
        
# with open("../files.json" , "r") as f:
#     file = json.load(f)

In [7]:
for games in tqdm(video_paths):
    # print(games)
    anno_path = normalize_path(os.path.join(games, 'Labels-v2.json'))
    annos = json_reader(anno_path)['annotations']
    # Bỏ qua các frames mà không được show
    # tức là các frame này khả năng cao không có ý nghĩa
    
    annos_v2 = [
        item for item in annos 
        if item['visibility'] != 'not shown' 
            # and item['label'] not in NO_NEEDED_CLASSES
    ]
    annos_v3 = [item for item in annos
        if item['label'] == 'Kick-off' and (item['gameTime'].startswith('1 - 00:') 
                                        or item['gameTime'].startswith('2 - 00:'))      
        ]
    annos_v2.extend(annos_v3)
    annos_v2 = sorted([
        dict(t) for t in {tuple(d.items()) for d in annos_v2}], 
        key = lambda x: x['gameTime']
        )
    break

  0%|          | 0/300 [00:00<?, ?it/s]

[{'gameTime': '1 - 00:00', 'label': 'Kick-off', 'position': '23', 'team': 'home', 'visibility': 'not shown'}, {'gameTime': '1 - 00:59', 'label': 'Ball out of play', 'position': '59880', 'team': 'not applicable', 'visibility': 'visible'}, {'gameTime': '1 - 01:23', 'label': 'Clearance', 'position': '83101', 'team': 'away', 'visibility': 'visible'}, {'gameTime': '1 - 03:19', 'label': 'Foul', 'position': '199234', 'team': 'home', 'visibility': 'visible'}, {'gameTime': '1 - 03:39', 'label': 'Ball out of play', 'position': '219536', 'team': 'not applicable', 'visibility': 'visible'}, {'gameTime': '1 - 04:02', 'label': 'Throw-in', 'position': '242488', 'team': 'home', 'visibility': 'visible'}, {'gameTime': '1 - 05:10', 'label': 'Corner', 'position': '310096', 'team': 'away', 'visibility': 'visible'}, {'gameTime': '1 - 06:51', 'label': 'Shots off target', 'position': '411199', 'team': 'home', 'visibility': 'visible'}, {'gameTime': '1 - 06:52', 'label': 'Ball out of play', 'position': '412175',

In [8]:
from datetime import datetime, timedelta

for games in tqdm(video_paths):
    # print(games)
    anno_path = normalize_path(os.path.join(games, 'Labels-v2.json'))
    annos = json_reader(anno_path)['annotations']
    # Bỏ qua các frames mà không được show
    # tức là các frame này khả năng cao không có ý nghĩa
    
    annos_v2 = [
        item for item in annos 
        if item['visibility'] != 'not shown' 
            # and item['label'] not in NO_NEEDED_CLASSES
    ]
    annos_v3 = [item for item in annos
        if item['label'] == 'Kick-off' and (item['gameTime'].startswith('1 - 00:') 
                                        or item['gameTime'].startswith('2 - 00:'))      
        ]
    annos_v2.extend(annos_v3)
    annos_v2 = sorted([
        dict(t) for t in {tuple(d.items()) for d in annos_v2}], 
        key = lambda x: x['gameTime']
        )
    
    # Find the end timestamp of the first half
    first_half_times = [item['gameTime'].split(' - ')[-1] for item in annos_v2 if item['gameTime'].startswith("1")]
    if first_half_times:
        first_half_end_time = max(first_half_times, key=lambda x: datetime.strptime(x, "%M:%S"))
        first_half_end_minutes = datetime.strptime(first_half_end_time, "%M:%S").hour * 60 + datetime.strptime(first_half_end_time, "%M:%S").minute
    else:
        first_half_end_minutes = 45  # Default to 45 minutes if no first half data

    # Group items
    grouped_items = defaultdict(list)
    for item in annos_v2:
        time_str = item['gameTime'].split(' - ')[-1]
        if item['gameTime'].startswith("1"):
            group = round_down_to_minute(time_str)
        else:
            group = round_down_to_minute_v2(time_str, first_half_end_minutes)
        grouped_items[group].append(item['label'])
        # grouped_items[group2].append(item['label'])
    # print(grouped_items)
    grouped_items = defaultdict(list, {k: v for k, v in grouped_items.items() if not k.startswith("-")})
    # sorted keys chỉ chứa các thông tin về các phút chứa sự kiện
    # tức là có thể có CÁC PHÚT không có sự kiện nào xảy ra
    # sort_keys là các time-minute 
    sorted_keys = sorted(
            grouped_items.keys(), 
            key = lambda x: int(x.split(":")[0])
        )
    result = {}
    previous_minute = None
    
    for key in sorted_keys:
        current_minute = int(key.split(":")[0])
        if previous_minute is not None and current_minute > previous_minute +1:
            for missing_minute in range(previous_minute + 1, current_minute):
                # Xử lý trường hợp có phút bị khuyết thì bổ sung thêm
                # Và đặt nó thành list rỗng
                result[f"{missing_minute:02}:00"] = ["Not Event"]      
        result[key] = grouped_items[key]
        previous_minute = current_minute
    #
    result = {k: list(set(v)) for k, v in result.items()}
    
    df = pandas.DataFrame({
        'chunk': [int(i.split(":")[0]) for i in result.keys()],
        'video_base_dir': games,
        "timestamp": result.keys(),
        "labels": [values for values in result.values()],  # Join list items into a single string
    })
    
    df['labels_encoder'] = df['labels'].apply(
        lambda label_list: ", ".join(
            str(v) for k, v in EVENT_DICT.items() if k in label_list
        ) if label_list else str(list(EVENT_DICT.values())[-1])
    )
    def get_video_path(row):
        minute = int(row['timestamp'].split(':')[0])
        return glob.glob(f"{row['video_base_dir']}/chunk_{minute}/*.mp4")

    df['video_path'] = df.apply(get_video_path, axis=1)
    
    def get_audio_path(row):
        minute = int(row['timestamp'].split(':')[0])
        return glob.glob(f"{row['video_base_dir']}/chunk_{minute}/*.mp3")

    df['audio_path'] = df.apply(get_audio_path, axis=1)

    df = df[df['video_path'].apply(lambda x: len(x) > 0) & df['audio_path'].apply(lambda x: len(x) > 0)]
    df.to_csv(f"{games}/{games.split('/')[-1]}.csv", index=False)

100%|██████████| 300/300 [02:57<00:00,  1.69it/s]


In [9]:
import os
import pandas
import csv
import shutil
from tqdm import tqdm
from src.preprocessing.video import normalize_path
basedir = "F:/video_classification"
ass = 0
n_chunks_to_remove =[]
n_path_to_remove=  []
for vdo in tqdm(os.listdir(basedir)):
    vdo_path = normalize_path(os.path.join(basedir, vdo))

    count_chunk = len([i for i in os.listdir(vdo_path) if os.path.isdir(f"{vdo_path}/{i}")]) 
    count_csv = len(pandas.read_csv(f"{vdo_path}/{vdo}.csv", sep = ","))
    if count_chunk <= 20:
        ass += 1
        shutil.rmtree(vdo_path)
    elif count_chunk > count_csv:
        n_chunks_to_remove.append(vdo_path)
        # count_remover = (count_chunk) - (count_csv)
        # for i in range(count_remover+1):
        #     if os.path.exists(f"{vdo_path}/chunk_{count_csv + i}"):
        #         shutil.rmtree(f"{vdo_path}/chunk_{count_csv + i}")
    elif count_chunk < count_csv:
        n_path_to_remove.append(vdo_path)
        # count_remover = count_csv - count_chunk
        # readcsv = pandas.read_csv(f"{vdo_path}/{vdo}.csv", sep = ",")
        # readcsv = readcsv[:-count_remover]
        # readcsv.to_csv(f"{vdo_path}/{vdo}.csv", index=False)

100%|██████████| 300/300 [00:17<00:00, 16.87it/s]


In [10]:

basedir = "F:/video_classification"
csv_files = [f"{basedir}/{i}/{i}.csv" for i in os.listdir(basedir)]

import pandas as pd
import glob

# Read and concatenate all CSVs
df2 = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)
# df2 = df2[~df2['video_path'].apply(lambda x: x == [])]

video_ids = []
for i in df2['video_path']:
    ites = str(i).replace('[', '').replace(']', '')
    ites = ites.replace("'", '').replace("'", '')
    video_ids.append(normalize_path(ites))
df2['video_id'] = video_ids

audio_ids = []
for i in df2['audio_path']:
    ites = str(i).replace('[', '').replace(']', '')
    ites = ites.replace("'", '').replace("'", '')
    audio_ids.append(normalize_path(ites))
df2['audio_id'] = audio_ids

df2 = df2[['chunk', 'video_base_dir', 'video_id', 'audio_id', 'labels_encoder']].rename(columns={'labels_encoder': 'label'})
# # Save the merged CSV
df2.to_csv("D:/projects/v2v/v5/data/base_dataset.csv", index=False, sep = ";")


In [21]:
event_dataset = pandas.read_csv("D:/projects/v2v/v5/data/base_dataset.csv", sep = ";")

event_dataset = event_dataset[['video_id','label']]
event_dataset.to_csv("D:/projects/v2v/v5/data/event_dataset.csv", index=False, sep = ";")

In [12]:
audio_ids = pandas.read_csv("D:/projects/v2v/v5/data/base_dataset.csv", sep = ";")['audio_id'].tolist()  # Chuyển thành list

# Lưu vào file JSON
with open("D:/projects/v2v/v5/data/audio_list.json", "w") as f:
    json.dump(audio_ids, f, indent=4)

In [13]:
import os
import shutil
import pandas as pd

def remove_incomplete_chunks(csv_path):
    df = pd.read_csv(csv_path, sep=';')
    removed_count = 0
    removed_chunks = set()

    for idx, row in df.iterrows():
        video_path = row['video_id']
        chunk_dir = os.path.dirname(video_path)

        # Bỏ qua nếu chunk đã bị xóa trước đó
        if not os.path.exists(chunk_dir) or chunk_dir in removed_chunks:
            continue

        frame_dir = os.path.join(chunk_dir, 'frames')
        if not os.path.exists(frame_dir):
            print(f"[WARNING] Frame folder not found: {frame_dir}")
            continue

        # Đếm số lượng ảnh
        image_count = len([
            f for f in os.listdir(frame_dir)
            if f.lower().endswith(('.jpg', '.jpeg', '.png'))
        ])

        if image_count < 30:
            print(f"[REMOVE] {chunk_dir} — only {image_count} frames")
            shutil.rmtree(chunk_dir)
            removed_chunks.add(chunk_dir)
            removed_count += 1

    print(f"\nTotal removed folders: {removed_count}")
    return removed_count

# Gọi hàm
csv_file = './data/event_dataset.csv'
remove_incomplete_chunks(csv_file)

FileNotFoundError: [Errno 2] No such file or directory: './data/event_dataset.csv'